In [1]:
import glob
import math
import sys
import timeit

sys.path.insert(0, '../../')
from visualize_dataset import *
from losses import *
from metrics import *
from predict import *
from train import *
from utils import *

Using TensorFlow backend.


In [2]:
path_train_images = '../../../data/stanford_t2_cropped/train/Slice/'
path_train_annotations = '../../../data/stanford_t2_cropped/train/ROI/'
path_train_predictions = './predictions.50e/train/'

path_val_images = '../../../data/stanford_t2_cropped/val/Slice/'
path_val_annotations = '../../../data/stanford_t2_cropped/val/ROI/'
path_val_predictions = './predictions.50e/val/'

path_test_images = '../../../data/stanford_t2_cropped/test/Slice/'
path_test_annotations = '../../../data/stanford_t2_cropped/test/ROI/'
path_test_predictions = './predictions.50e/test/'

path_save_weights = './weights.50e/'
path_load_weights = None

train_vgg = False
path_vgg_weights = '../../../data/vgg16_weights_th_dim_ordering_th_kernels.h5'

n_classes = 2
input_height = 224
input_width = 224
n_channel_annotations = 1

In [3]:
print('Number train images: ', len(glob.glob(path_train_images + '*.png')))
print('Number val images: ', len(glob.glob(path_val_images + '*.png')))
print('Number test images: ', len(glob.glob(path_test_images + '*.png')))

Number train images:  728
Number val images:  36
Number test images:  35


In [4]:
model_name = 'vgg_unet_padded'
epochs = 50
train_batch_size = 8
val_batch_size = len(glob.glob(path_val_images + '*.png'))

In [5]:
optimizer = 'adadelta'
metrics = ['accuracy', dice, iou]

class_props = calc_class_proportions(path_train_annotations, n_classes, n_channel_annotations)
class_weights = calc_class_weights(path_train_annotations, n_classes, n_channel_annotations)
class_weights_log = calc_class_weights(path_train_annotations, n_classes, n_channel_annotations, scale = 'log')

print(class_props)
print(class_weights)
print(class_weights_log)

loss = weighted_categorical_crossentropy_loss(class_weights_log)

[0.98302371 0.01697629]
[ 1.         57.90569446]
[0.01712204 4.07593777]


In [ ]:
start = timeit.default_timer()
train(
    path_train_images, path_train_annotations, train_batch_size, 
    path_val_images, path_val_annotations, val_batch_size,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights, path_vgg_weights = path_vgg_weights, train_vgg = train_vgg, path_load_weights = None
)
end = timeit.default_timer()

Model output shape (None, 50176, 2)
Epoch 0
Epoch 1/1
91/91 [==============================] - 582s 6s/step - loss: 0.0480 - acc: 0.9592 - dice: 9.5071e-04 - iou: 4.8235e-04 - val_loss: 0.0382 - val_acc: 0.9850 - val_dice: 0.0000e+00 - val_iou: 0.0000e+00
Epoch 1
Epoch 1/1
91/91 [==============================] - 575s 6s/step - loss: 0.0356 - acc: 0.7551 - dice: 0.0020 - iou: 0.0010 - val_loss: 0.0284 - val_acc: 0.3949 - val_dice: 0.0021 - val_iou: 0.0011
Epoch 2
Epoch 1/1
91/91 [==============================] - 575s 6s/step - loss: 0.0290 - acc: 0.4908 - dice: 0.0019 - iou: 9.7072e-04 - val_loss: 0.0286 - val_acc: 0.5056 - val_dice: 5.7975e-04 - val_iou: 2.8996e-04
Epoch 3
Epoch 1/1
91/91 [==============================] - 574s 6s/step - loss: 0.0262 - acc: 0.4154 - dice: 0.0016 - iou: 8.1147e-04 - val_loss: 0.0214 - val_acc: 0.2749 - val_dice: 0.0020 - val_iou: 0.0010
Epoch 4
Epoch 1/1
91/91 [==============================] - 576s 6s/step - loss: 0.0235 - acc: 0.3748 - dice: 0.0012 

In [ ]:
print('Total training time: ', end - start)

In [6]:
epochs = 40

In [7]:
predict(
    path_train_images, path_train_annotations, path_train_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics, 
    path_save_weights
)

              precision    recall  f1-score   support

           0       0.45      0.01      0.03  35908016
           1       0.00      0.01      0.00    620112

   micro avg       0.01      0.01      0.01  36528128
   macro avg       0.22      0.01      0.01  36528128
weighted avg       0.44      0.01      0.03  36528128

F1/dice (binary):  0.0001960716609723356
Jaccard/IOU:  9.8045442452544e-05


In [ ]:
visualize_dataset(
    path_train_images, path_train_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations,
    dir_predictions = path_train_predictions
)

In [9]:
predict(
    path_val_images, path_val_annotations, path_val_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights
)

              precision    recall  f1-score   support

           0       0.83      0.01      0.02   1779279
           1       0.01      0.85      0.03     27057

   micro avg       0.02      0.02      0.02   1806336
   macro avg       0.42      0.43      0.02   1806336
weighted avg       0.81      0.02      0.02   1806336

F1/dice (binary):  0.0255281831119466
Jaccard/IOU:  0.012929120027745614


In [ ]:
visualize_dataset(
    path_val_images, path_val_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations,
    dir_predictions = path_val_predictions
)

In [11]:
predict(
    path_test_images, path_test_annotations, path_test_predictions,
    n_classes, input_width, input_height, 
    model_name, epochs, loss, optimizer, metrics,
    path_save_weights
)

              precision    recall  f1-score   support

           0       0.55      0.01      0.02   1714591
           1       0.01      0.62      0.03     41569

   micro avg       0.03      0.03      0.03   1756160
   macro avg       0.28      0.31      0.03   1756160
weighted avg       0.53      0.03      0.02   1756160

F1/dice (binary):  0.029058654591894918
Jaccard/IOU:  0.0147435410290599


In [ ]:
visualize_dataset(
    path_test_images, path_test_annotations, 
    n_classes, n_channel_annotations = n_channel_annotations, 
    dir_predictions = path_test_predictions
)